In [ ]:
!pip install datasets faiss-cpu langchain duckduckgo-search wikipedia yfinance sentence-transformers requests transformers


In [ ]:
!pip install -U langchain-community  tf-keras

In [ ]:
!pip uninstall keras -y

In [ ]:
!pip list | grep -E "tensorflow|keras"

🧠 Step 1: Setup Ollama + vLLM LLM Wrappers

In [2]:
import requests
from typing import List, Optional
from langchain.llms.base import LLM

import sys
import tf_keras as keras
sys.modules["keras"] = keras

# Now it's safe to import transformers
from transformers import TFAutoModel, AutoTokenizer
from typing import Optional, List
from langchain.llms.base import LLM
from pydantic import BaseModel, Field
import requests

class OllamaLLM(LLM):
    model: str = "hf.co/scb10x/llama3.2-typhoon2-t1-3b-research-preview-gguf:latest"
    endpoint: str = "http://172.16.30.146:11434/api/generate"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        payload = {
            "model": self.model,
            "prompt": prompt,
            "stream": False
        }

        try:
            headers = {"Content-Type": "application/json"}
            response = requests.post(self.endpoint, json=payload, headers=headers)
            response.raise_for_status()

            data = response.json()
            if "response" not in data:
                raise ValueError(f"No 'response' in Ollama result: {data}")
            
            return data["response"]
        except Exception as e:
            print(f"❌ Error in OllamaLLM: {e}")
            return ""

    @property
    def _llm_type(self) -> str:
        return "ollama"

# --- Instantiate Two Models ---

model_chat = "hf.co/scb10x/typhoon2.1-gemma3-4b-gguf:latest"
model_reasoning = "hf.co/scb10x/llama3.2-typhoon2-t1-3b-research-preview-gguf:latest"

llm_chat = OllamaLLM(model=model_chat)
llm_reasoning = OllamaLLM(model=model_reasoning)


# --- Optional: Helper Function to Use Reasoning Model ---
def get_reasoning_answer(prompt: str) -> str:
    return llm_reasoning(prompt)

2025-06-25 22:32:07.900166: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-25 22:32:07.912751: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750890727.926042  371739 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750890727.929788  371739 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750890727.940139  371739 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

📚 Step 2: Vector Store Setup (unchanged)

In [3]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from datasets import load_dataset, load_from_disk

# Load datasets (subset for demo)
#data1 = load_dataset('Josephgflowers/Finance-Instruct-500k', split='train[:2000]')
#data2 = load_dataset("airesearch/WangchanThaiInstruct", split='train[:2000]')
#data3 = load_dataset('PowerInfer/LONGCOT-Refine-500K', split='train[:2000]')
#data1.save_to_disk('finance_instruct_500k')
#data2.save_to_disk('WangchanThaiInstruct')
#data3.save_to_disk('longcot-refine-500k')



data1 = load_from_disk("finance_instruct_500k")
data2 = load_from_disk("WangchanThaiInstruct")
data3 = load_from_disk("longcot-refine-500k")


def extract_text(dataset, fields):
    rows = []
    for sample in dataset:
        text = ""
        for field in fields:
            if field in sample and sample[field] and str(sample[field]).strip():
                text += f"{field}: {sample[field]}\n"
        if text.strip():
            rows.append(Document(page_content=text.strip()))
    return rows

# Apply correct field names for each dataset
docs1 = extract_text(data1, fields=['system', 'user', 'assistant'])
docs2 = extract_text(data2, fields=['Instruction', 'Input', 'Output'])
docs3 = extract_text(data3, fields=['prompt', 'response'])
# ✅ Extract and combine documents
docs = docs1 + docs2 + docs3
print(f"Loaded {len(docs)} documents")

# ✅ Split the documents into smaller chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = splitter.split_documents(docs)
print(f"Split into {len(split_docs)} chunks")

# ✅ Embed and save
texts = [doc.page_content for doc in split_docs]
if not texts:
    raise ValueError("No texts to embed — check if documents were loaded and split correctly")

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(split_docs, embedding_model)
#vectorstore.save_local("finance_rag_faiss102")
print("Vectorstore saved successfully.")


Loaded 6000 documents
Split into 85723 chunks


/tmp/ipykernel_371739/913936021.py:51: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


Vectorstore saved successfully.


🔁 Step 3: RAG Chain and Fallback Logic

In [4]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from datasets import load_dataset

from langchain.chains import RetrievalQA
from duckduckgo_search import DDGS
import wikipedia
import yfinance as yf

# Instantiate Embedding Model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Instantiate Ollama LLMs (chat and reasoning)
model_chat = "hf.co/scb10x/typhoon2.1-gemma3-4b-gguf:latest"
model_reasoning = "hf.co/scb10x/llama3.2-typhoon2-t1-3b-research-preview-gguf:latest"

llm_chat = OllamaLLM(model=model_chat)
llm_reasoning = OllamaLLM(model=model_reasoning)

# Load retriever and build QA chain using chat model
retriever = FAISS.load_local(
    "finance_rag_faiss102",
    embedding_model,
    allow_dangerous_deserialization=True
).as_retriever()

qa = RetrievalQA.from_chain_type(llm=llm_chat, retriever=retriever)



# Fallback tools
def duckduckgo_search(query, max_results=3):
    with DDGS() as ddgs:
        return [r["title"] + " - " + r["href"] for r in ddgs.text(query, max_results=max_results)]

def wikipedia_summary(query):
    try:
        return wikipedia.summary(query, sentences=2)
    except:
        return "No Wikipedia summary found."

def get_stock_data(ticker):
    try:
        stock = yf.Ticker(ticker)
        return stock.info.get('longBusinessSummary', 'No stock summary found.')
    except:
        return "Error loading stock."

Step 4: Run the Final QA Function

In [5]:
# Self-confidence rating using reasoning model
def is_confident_llm(llm, query, answer, threshold=7):
    if not answer or len(answer.strip()) < 30:
        return False

    prompt = f"""
Question: {query}

Answer: {answer}

On a scale from 1 to 10, how confident are you in this answer? Only respond with a number.
""".strip()

    try:
        rating_str = llm(prompt).strip()
        rating = int(''.join(filter(str.isdigit, rating_str)))  # safely extract digits
    except Exception as e:
        print(f"⚠️ Warning: Failed to get confidence rating from LLM: {e}")
        return False

    print(f"LLM self-confidence rating: {rating}")
    return rating >= threshold

In [6]:
# Main QA pipeline with fallback
def answer_question(query, use_reasoning=False):
    if use_reasoning:
        response = llm_reasoning(query)
        print("🧠 Reasoning Answer:\n", response)
        return response

    rag_answer = qa.run(query)
    print("💬 RAG Answer (via Ollama chat model):\n", rag_answer)

    if is_confident_llm(llm_reasoning, query, rag_answer):
        print("✅ Confident in RAG answer (via self-evaluation)")
        return rag_answer
    else:
        print("⚠️ Fallback triggered...")
        duck = duckduckgo_search(query)
        wiki = wikipedia_summary(query)
        fallback_str = f"RAG: {rag_answer}\nDuckDuckGo: {duck}\nWikipedia: {wiki}"
        return fallback_str

🧠 2. Load Questions

In [7]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA

# Define model names
model_chat = "hf.co/scb10x/typhoon2.1-gemma3-4b-gguf:latest"
model_reasoning = "hf.co/scb10x/llama3.2-typhoon2-t1-3b-research-preview-gguf:latest"

# Create Ollama LLMs
llm_chat = OllamaLLM(model=model_chat)
llm_reasoning = OllamaLLM(model=model_reasoning)

# Load FAISS vector store and retriever
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
retriever = FAISS.load_local(
    "finance_rag_faiss", embedding_model, allow_dangerous_deserialization=True
).as_retriever(search_kwargs={"k": 10})  # get top 10 for reranking

# Create RAG chain using the chat model
qa_chain = RetrievalQA.from_chain_type(llm=llm_reasoning, retriever=retriever)

🌍 Step 3: Deep Search Fallback (DuckDuckGo + Wikipedia)

In [8]:
from duckduckgo_search import DDGS
import wikipedia

def deep_search_summary(query):
    with DDGS() as ddgs:
        results = [r["body"] for r in ddgs.text(query, max_results=3)]
    try:
        wiki = wikipedia.summary(query, sentences=2)
    except:
        wiki = ""
    return "\n".join(results + [wiki])

📊 Step 4: Answer Generator with Confidence Check

In [9]:
from langchain.schema import Document

def rerank_documents(query: str, documents: list[Document], llm) -> list[Document]:
    context = "\n\n".join([f"Document {i+1}: {doc.page_content}" for i, doc in enumerate(documents)])
    prompt = (
        f"You are a helpful assistant that reranks documents by relevance to the question.\n\n"
        f"Question: {query}\n\n"
        f"{context}\n\n"
        f"Rank the top 3 most relevant documents. Respond only with the list of document numbers in order, e.g.: 2, 1, 3"
    )

    response = llm.predict(prompt)
    #print("Reranker response:", response)

    try:
        indices = [int(x.strip()) - 1 for x in response.split(",") if x.strip().isdigit()]
        return [documents[i] for i in indices if 0 <= i < len(documents)]
    except:
        return documents[:3]  # fallback

def generate_answer(query: str) -> str:
    try:
        # Step 1: Retrieve top-k documents
        retrieved_docs = retriever.get_relevant_documents(query)

        # Step 2: Rerank documents
        reranked_docs = rerank_documents(query, retrieved_docs, llm_reasoning)

        # Step 3: Create RAG prompt manually from reranked docs
        context = "\n\n".join([doc.page_content for doc in reranked_docs])
        prompt = f"""Answer the question using the context below. Be concise and accurate.

Context:
{context}

Question: {query}
Answer:"""

        rag_answer = llm_chat.predict(prompt)
        #print("DEBUG rag_answer:", rag_answer)

    except Exception as e:
        print(f"Error during reranked RAG: {e}")
        rag_answer = ""

    # Step 4: Confidence check using reasoning model
    if is_confident_llm(llm_chat, query, rag_answer):
        print("✅ Confident RAG answer")
        return rag_answer


    #return rag_answer
    # Step 5: Fallback to deep search if RAG not confident
    #print("⚠️ Low confidence in RAG — doing deep search...")
    #context = deep_search_summary(query)

    # Step 6: Ask reasoning model with external context
    fallback_prompt = f"""Given the following question and background info, choose the best option (A, B, C, D, Rise, or Fall). Do not explain.

Background: {context}
Question: {query}
Final answer:"""

    reasoning_answer = llm_reasoning(fallback_prompt)
    print("🧠 Reasoning Answer:", reasoning_answer)
    return reasoning_answer

📋 Step 5: Load Excel, Answer All, Save

In [10]:
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# Load data
df = pd.read_csv("test_wk.csv")  # adjust path
queries = df["query"].tolist()

# Valid answer types
valid_answers = ["A", "B", "C", "D", "RISE", "FALL"]

# Function to clean and extract valid answer
def process_query(q):
    try:
        answer = generate_answer(q)
        answer_clean = answer.strip().upper()

        # Extract first valid keyword
        for va in valid_answers:
            if va in answer_clean:
                return va
        return ""
    except Exception as e:
        print(f"Error processing query: {e}")
        return ""

# Run in parallel
results = []
with ThreadPoolExecutor(max_workers=4) as executor:  # adjust number of workers if needed
    future_to_query = {executor.submit(process_query, q): q for q in queries}
    for future in tqdm(as_completed(future_to_query), total=len(queries)):
        results.append(future.result())

# Save or display
df["predicted_answer"] = results
#print(df[["query", "predicted_answer"]])


/tmp/ipykernel_371739/101187831.py:24: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(query)
  0%|          | 0/499 [00:00<?, ?it/s]/tmp/ipykernel_371739/101187831.py:12: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm.predict(prompt)
/tmp/ipykernel_371739/101187831.py:64: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  reasoning_answer = llm_reasoning(fallback_prompt)
  0%|          | 1/499 [00:17<2:29:12, 17.98s/it]

🧠 Reasoning Answer: <thoughts>

<plan>
<step order="1">Identifying key details in the background and question</step>
<step order="2">Determining which option aligns best with the context of maximizing unique partnerships</step>
<step order="3">Selecting the most appropriate answer choice based on the given options</step>
</plan>

<step>
<title>Identifying key details in the background and question</title>
<scratch_pad>
- The focus is on leveraging corporate executive experience to boost economic growth.
- The main challenge mentioned is maximizing unique partnerships.
</scratch_pad>
<summary>I'm focusing on understanding that the goal is about creating partnerships for economic growth, which indicates an entrepreneurial mindset.</summary>
<next_step>Determining which option aligns best with the context of maximizing unique partnerships</next_step>
</step>

<step>
<title>Determining which option aligns best with the context of maximizing unique partnerships</title>
<scratch_pad>
- Optio

  0%|          | 2/499 [00:22<1:22:10,  9.92s/it]

🧠 Reasoning Answer: <thoughts>

<plan>
<step order="1">Understanding the background information</step>
<step order="2">Identifying key concepts related to ethical behavior</step>
<step order="3">Evaluating the provided options A, B, C, and D</step>
<step order="4">Selecting the most appropriate option based on the analysis</step>
</plan>

<step>
<title>Understanding the background information</title>
<scratch_pad>
The question revolves around how to effectively communicate ethical behavior within an organization. The COSO (Committee of Sponsoring Organizations of the Treadway Commission) framework emphasizes a comprehensive approach for effective risk management and communication of ethical standards.
</scratch_pad>
<summary>Understanding that ethical behavior must be communicated clearly and consistently across all levels of the organization to ensure alignment with organizational values.</summary>
<next_step>Identifying key concepts related to ethical behavior</next_step>
</step>

<s

  1%|          | 3/499 [00:24<51:34,  6.24s/it]  

...

🧠 Reasoning Answer: <thoughts>

<plan>
<step order="1">วิเคราะห์ข้อกำหนดของโจทย์</step>
<step order="2">ประเมินตัวเลือก A, B, C และ D สำหรับความเกี่ยวข้อง</step>
<step order="3">ระบุคำตอบที่ถูกต้องตามข้อมูลที่ให้มา</step>
</plan>

<step>
<title>วิเคราะห์ข้อกำหนดของโจทย์</title>
<scratch_pad>
เรากำลังมองหาวิธีการส่งมอบผลิตภัณฑ์ที่ตรงกับความต้องการของผู้บริโภค ซึ่งหมายความว่าต้องเกี่ยวข้องกับการเข้าใจและตอบสนองความต้องการที่มีอยู่แล้ว ตัวเลือกจะถูกประเมินตามนี้
</scratch_pad>
<summary>ผมกำลังทำความเข้าใจคำถามเกี่ยวกับกระบวนการส่งมอบผลิตภัณฑ์ให้ตรงตามความต้องการของผู้บริโภคและวิธีการที่ตัวเลือก A, B, C และ D แสดงออกมา</summary>
<next_step>ประเมินตัวเลือก A, B, C และ D สำหรับความเกี่ยวข้อง</next_step>
</step>

<step>
<title>ประเมินตัวเลือก A, B, C และ D สำหรับความเกี่ยวข้อง</title>
<scratch_pad>
- ตัวเลือก A: แนวคิดผลิตภัณฑ์ใหม่และการปรับปรุง - เกี่ยวข้องกับการสร้างความพึงพอใจและตอบสนองต่อผู้บริโภคโดยตรง
- ตัวเลือก B: การขาย - ไม่ใช่วิธีการส่งมอบ แต่เป็นกระบวนการหลังที่เกิดขึ้นจากการเข้าใจ

100%|█████████▉| 498/499 [56:29<00:05,  5.37s/it]

🧠 Reasoning Answer: <thoughts>

<plan>
<step order="1">Reviewing the context</step>
<step order="2">Identifying the concept being measured</step>
<step order="3">Selecting the appropriate option from the given choices</step>
</plan>

<step>
<title>Reviewing the context</title>
<scratch_pad>
The question asks about a specific measure of price and quantity in relation to demand.
</scratch_pad>
<summary>I'm analyzing what aspect of price and quantity is being referenced in the question.</summary>
<next_step>Identifying the concept being measured</next_step>
</step>

<step>
<title>Identifying the concept being measured</title>
<scratch_pad>
The phrase "price times quantity" suggests we are looking at a product that represents how much price and quantity interact.
This can be interpreted as total revenue, which is calculated by multiplying price by quantity sold.
</scratch_pad>
<summary>The measure of interest appears to relate to the relationship between price and demand, indicating it mig

100%|██████████| 499/499 [56:30<00:00,  6.79s/it]

🧠 Reasoning Answer: <thoughts>

<plan>
<step order="1">Analyzing context</step>
<step order="2">Identifying key indicators</step>
<step order="3">Assessing market trends</step>
<step order="4">Determining the direction of price movement</step>
<step order="5">Selecting appropriate response option</step>
</plan>

<step>
<title>Analyzing context</title>
<scratch_pad>
The provided data and tweets refer to stock performance, particularly the closing price of $bac on 2017-12-13. Key details include:
- Previous day's high: 0.9
- Previous day's low: -1.6
- Previous day's close: 2.3
- Volume of trades mentioned is significant, suggesting active trading.
</scratch_pad>
<summary>I've reviewed the data and context to understand how they relate to the closing price of $bac on December 13th.</summary>
<next_step>Identifying key indicators</next_step>
</step>

<step>
<title>Identifying key indicators</title>
<scratch_pad>
The tweets mention active trading and significant movements in stock prices. T

In [ ]:
df

In [11]:
#df["answer"] = answers
df.to_csv("answered_questions103.csv", index=False)

In [34]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
ollama_llm = OllamaLLM(model=model_reasoning)
print(ollama_llm("What is the capital of Thailand?"))
print(model_reasoning)

❌ Error in OllamaLLM: 404 Client Error: Not Found for url: http://172.16.30.146:11434/api/generate

hf.co/scb10x/llama3.2-typhoon2-t1-3b-research-preview-gguf:latest


In [13]:
!curl http://172.16.30.146:11434/api/tags

{"models":[{"name":"hf.co/scb10x/llama3.2-typhoon2-t1-3b-research-preview-gguf:latest","model":"hf.co/scb10x/llama3.2-typhoon2-t1-3b-research-preview-gguf:latest","modified_at":"2025-06-25T12:47:49.787578009Z","size":2019378094,"digest":"56b1b463a6bcce24ee938bd2b3abdb81686602f6fb6de3a54030ced8dac56a44","details":{"parent_model":"","format":"gguf","family":"llama","families":["llama"],"parameter_size":"3.21B","quantization_level":"unknown"}},{"name":"hf.co/scb10x/typhoon2.1-gemma3-4b-gguf:latest","model":"hf.co/scb10x/typhoon2.1-gemma3-4b-gguf:latest","modified_at":"2025-06-25T09:04:49.1355109Z","size":2489765421,"digest":"37354827f7b37040c83230a9b0252a803408f3a4d7511561a5457c1eb70c045e","details":{"parent_model":"","format":"gguf","family":"gemma3","families":["gemma3"],"parameter_size":"3.88B","quantization_level":"unknown"}},{"name":"llama3.2:3b","model":"llama3.2:3b","modified_at":"2025-06-25T09:00:32.26182376Z","size":2019393189,"digest":"a80c4f17acd55265feec403c7aef86be0c25983ab27

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [1]:
!curl -X POST http://172.16.30.146:11434/api/generate -d '{ "model": "hf.co/scb10x/llama3.2-typhoon2-t1-3b-research-preview-gguf:latest",  "prompt": "What is the capital of Thailand?",  "stream": false}'

{"model":"hf.co/scb10x/llama3.2-typhoon2-t1-3b-research-preview-gguf:latest","created_at":"2025-06-25T22:31:57.812055091Z","response":"\u003cthoughts\u003e\n\n\u003cplan\u003e\n\u003cstep order=\"1\"\u003eIdentify the user's request\u003c/step\u003e\n\u003cstep order=\"2\"\u003eConfirm the capital city of Thailand\u003c/step\u003e\n\u003cstep order=\"3\"\u003eProvide additional context if necessary\u003c/step\u003e\n\u003c/plan\u003e\n\n\u003cstep\u003e\n\u003ctitle\u003eIdentify the user's request\u003c/title\u003e\n\u003cscratch_pad\u003e\nThe user is asking specifically for the capital city of Thailand.\n\u003c/scratch_pad\u003e\n\u003csummary\u003eThe user wants to know the capital of Thailand.\u003c/summary\u003e\n\u003cnext_step\u003eConfirm the capital city of Thailand\u003c/next_step\u003e\n\u003c/step\u003e\n\n\u003cstep\u003e\n\u003ctitle\u003eConfirm the capital city of Thailand\u003c/title\u003e\n\u003cscratch_pad\u003e\nThe capital of Thailand is indeed Bangkok. \n- Locate

In [36]:
payload = {
    "model": "hf.co/scb10x/llama3.2-typhoon2-t1-3b-research-preview-gguf:latest",
    "prompt": "What is the capital of Thailand?",
    "stream": False
}
response = requests.post('http://172.16.30.146:11434/api/generate', json=payload)

In [ ]:
results